In [2]:
import numpy as np
import json
import sys
sys.path.append('./src')
import data
import torch
import training


import openai
openai.api_key = open('key.txt').read().strip()

%load_ext autoreload
%autoreload 2

In [3]:
train_dataset = data.FOL2NL(split='train')
dev_dataset = data.FOL2NL(split='dev')
print(len(train_dataset), len(dev_dataset))

5233 1285


In [5]:
# fol = '∀x (happy(x) → ¬isred(x))'
# fol = '∀x (happy(x) → isred(x))'
# fol = '∃x (happy(x) ∧ isred(x))'
# fol = '∃x (happy(x) ∧ ¬isred(x))'


prompt = fol + ' ->'
print(prompt)
response = openai.Completion.create(
    model='curie:ft-personal-2022-12-06-07-10-39',
    prompt=prompt,
    stop=["\n"],
    temperature=0
    )
text = response['choices'][0]['text']
print(text)

∀x (happy(x) → isred(x)) ->
 If a cartoon character is happy, then it is red.


In [ ]:

fol = '∀x(smile(x) → happy(x))'
prompt = fol + ' ->'
print(prompt)
response = openai.Completion.create(
    model='curie:ft-personal-2022-12-06-07-10-39',
    prompt=prompt,
    stop=["\n"],
    temperature=0.7
    )
text = response['choices'][0]['text']
print(text)
print(nl)
print('----')

In [7]:

for fol, nl in dev_dataset[234:244]:
    prompt = fol + ' ->'
    print(prompt)
    response = openai.Completion.create(
        model='curie:ft-personal-2022-12-06-07-10-39',
        prompt=prompt,
        stop=["\n"],
        temperature=0.7
        )
    text = response['choices'][0]['text']
    print(text)
    print(nl)
    print('----')

Includes(diamondmine, creedbrothers) ∧ Includes(diamondmine, ivynile) ->
 Diamond Mine includes the Creed Brothers, Ivy Nile.
Diamond Mine includes the Creed Brothers, and Ivy Nile.
----
HasFeud(imperium, diamondmine) ->
 Imperium has a feud with Diamond Mine.
Imperium has a feud with Diamond Mine.
----
Leads(roderickstrong, creedbrothers) ->
 Roderick strong leads the Creed Brothers.
Roderick strong leads the Creed Brothers.
----
ProfessionalWrestlingStable(diamondmine) ∧ FormedIn(diamondmine, wwe) ->
 Diamond Mine is a professional wrestling stable, formed in WWE.
Diamond Mine is a professional wrestling stable, formed in WWE.
----
Leads(roderickstrong, diamondmine) ->
 Roderick strong leads Diamond Mine.
Roderick Strong leads Diamond Mine.
----
Includes(diamondmine, creedbrothers) ∧ Includes(diamondmine, ivynile) ->
 Diamond Mine includes the Creed Brothers, and Ivy Nile.
Diamond Mine includes the Creed Brothers, and Ivy Nile.
----
HasFeud(imperium, diamondmine) ->
 Imperium has a f

In [42]:
unique_sample = set()
with open('dataset/folio/gpt-finetune.jsonl','w',encoding='utf-8') as f:
    for x,y in train_dataset:
        if (x,y) in unique_sample:
            continue
        f.write(json.dumps({'prompt':x,'completion':y})+'\n')
        unique_sample.add((x,y))

    for x,y in dev_dataset:
        if (x,y) in unique_sample:
            continue
        f.write(json.dumps({'prompt':x,'completion':y})+'\n')
        unique_sample.add((x,y))

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
for char in train_dataset.speicial_tokens:
    print(tokenizer.convert_ids_to_tokens(tokenizer.encode(char)))

['âĨĴ']
['âĨ', 'Ķ']
['Â', '¬']
['â', 'Ĭ', 'ķ']
['âĪ', '¨']
['âĪ', '§']
['âĪ', 'Ģ']
['âĪ', 'ĥ']


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer.add_tokens(train_dataset.speicial_tokens)
model.resize_token_embeddings(len(tokenizer))

c:\Users\cheny\Anaconda3\envs\wfsa\lib\site-packages\transformers\models\t5\tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Embedding(32108, 768)

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=training.collate_fn(tokenizer))
test_loder = torch.utils.data.DataLoader(dev_dataset, batch_size=8, shuffle=True, collate_fn=training.collate_fn(tokenizer))

In [ ]:
training.train(model, tokenizer, train_loader, test_loder, epoch=20, update_every=16)